In [1]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

sys.path.append("/global/homes/r/rtojeiro/SV/python")
import VI_utils as VI

import matplotlib.pyplot as plt 

pd.set_option('display.max_rows', 5)

In [2]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/'
tiles = ['66003']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']


In [3]:
VI_dir = os.environ['HOME']+'/SV/VI_files/SV0/BGS/'
save_filename = 'truth_table_BGS_v1.1.csv'

### Read individual VI files and concatenate all "VI Issue" fields by TargetID

In [4]:
vi = VI.read_individual_VI(VI_dir)

Reading all individual VI files in /global/homes/r/rtojeiro/SV/VI_files/SV0/BGS/
desi-vi_SV0_BGS_tile66003_night20200315_27_IAN.csv
desi-vi_SV0_BGS_tile66003_night20200315_17_ARW.csv
desi-vi_SV0_BGS_tile66003_night20200315_42_ARW.csv
desi-vi_SV0_BGS_tile66003_night20200315_10_BJW.csv
desi-vi_SV0_BGS_tile66003_night20200315_1_ARW.csv
desi-vi_SV0_BGS_tile66003_night20200315_19_SMC.csv
desi-vi_SV0_BGS_tile66003_night20200315_21_acedge.csv
desi-vi_SV0_BGS_tile66003_night20200315_30_ORM.csv
desi-vi_SV0_BGS_tile66003_night20200315_21_ORM.csv
desi-vi_SV0_BGS_tile66003_night20200315_15_acedge.csv
desi-vi_SV0_BGS_tile66003_night20200315_12_ARW.csv
desi-vi_SV0_BGS_tile66003_night20200315_26_IAN.csv
desi-vi_SV0_BGS_tile66003_night20200315_35_acedge.csv
desi-vi_SV0_BGS_tile66003_night20200315_9_PSZ.csv
desi-vi_SV0_BGS_tile66003_night20200315_19_CMH.csv
desi-vi_SV0_BGS_tile66003_night20200315_29_IAN.csv
desi-vi_SV0_BGS_tile66003_night20200315_3_CMH.csv
desi-vi_SV0_BGS_tile66003_night20200315_32_ace

In [5]:
vi

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment
0,35191331848455662,-1,0.33.0.dev3890,0,GALAXY,0.3908,SMC,4,--,--,--,--
1,35191331848455971,-1,0.33.0.dev3890,0,GALAXY,0.4353,SMC,3,--,--,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...
7743,35191341470191058,-1,0.33.0.dev3890,0,GALAXY,0.3564,CMH,4,--,--,--,--
7744,35191341470191355,-1,0.33.0.dev3890,0,GALAXY,0.2524,CMH,4,--,--,--,--


In [6]:
#create new column - concatenates all entries in the "VI issue" field
vi['all VI issue'] = vi.groupby('TargetID')['VI issue'].transform(lambda x: '|'.join(x))

In [7]:
vi

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,all VI issue
0,35191331848455662,-1,0.33.0.dev3890,0,GALAXY,0.3908,SMC,4,--,--,--,--,--|--|--|--
1,35191331848455971,-1,0.33.0.dev3890,0,GALAXY,0.4353,SMC,3,--,--,--,--,--|--|C|--
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7743,35191341470191058,-1,0.33.0.dev3890,0,GALAXY,0.3564,CMH,4,--,--,--,--,--|--|--
7744,35191341470191355,-1,0.33.0.dev3890,0,GALAXY,0.2524,CMH,4,--,--,--,--,--|--|--


In [8]:
#add new column, with all comments concatenated
vi['all VI comments'] = vi.groupby('TargetID')['VI comment'].transform(lambda x: '|'.join(x))

In [9]:
vi

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,all VI issue,all VI comments
0,35191331848455662,-1,0.33.0.dev3890,0,GALAXY,0.3908,SMC,4,--,--,--,--,--|--|--|--,"--|--|Mg1 absorption"","" don't see much else|LRG"
1,35191331848455971,-1,0.33.0.dev3890,0,GALAXY,0.4353,SMC,3,--,--,--,--,--|--|C|--,--|--| I am not very confident in the redshift...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7743,35191341470191058,-1,0.33.0.dev3890,0,GALAXY,0.3564,CMH,4,--,--,--,--,--|--|--,--|--|--
7744,35191341470191355,-1,0.33.0.dev3890,0,GALAXY,0.2524,CMH,4,--,--,--,--,--|--|--,--|LRG|--


### Make truth table

Read all the partial truth table files and concatenate into one

In [10]:
filenames = ['truth_table_BGS_sets16_30_wcomments.txt', 'truth_table_BGS_sets1_15_wcomments.txt', 'truth_table_BGS_sets31_45_wcomments.txt']

truth_table = pd.read_csv(VI_dir+filenames[0])
for i in range(1,len(filenames)):
    partial_table = pd.read_csv(VI_dir+filenames[i])
    truth_table = truth_table.append(partial_table, ignore_index=True,sort=False)


In [11]:
truth_table

,TargetID,Redrock z,Redrock spectype,best redshift,best spectype,vi_combined_flag,N_VI,DELTACHI2,FIBER,FLUX_G,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,merger comment
0,35191341453413620,0.3855,GALAXY,0.3855,GALAXY,4.0,3,651.904116,1252,2.533373,12.313652,27.995237,1.009937,4.908875,11.160387,0.941849,0.960447,0.977679,none
1,35191341453413884,0.1129,GALAXY,0.1129,GALAXY,4.0,3,2426.963718,1407,12.787602,29.821730,50.967160,6.675207,15.567125,26.605167,0.929344,0.951838,0.972768,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2239,35191366115918216,0.3193,GALAXY,0.3193,GALAXY,4.0,4,1195.571405,2592,5.065198,10.414370,16.984789,0.958967,1.971696,3.215638,0.957088,0.970887,0.983610,none
2240,35191366115918301,0.4056,GALAXY,0.4056,GALAXY,4.0,4,944.793370,2568,3.731254,19.783285,47.370037,1.259283,6.676778,15.987193,0.956837,0.970716,0.983513,none


### Add the 'all VI issue' field to the truth table

In [12]:
vi[['TargetID', 'all VI issue', 'all VI comments']]

,TargetID,all VI issue,all VI comments
0,35191331848455662,--|--|--|--,"--|--|Mg1 absorption"","" don't see much else|LRG"
1,35191331848455971,--|--|C|--,--|--| I am not very confident in the redshift...
...,...,...,...
7743,35191341470191058,--|--|--,--|--|--
7744,35191341470191355,--|--|--,--|LRG|--


In [16]:
vi.groupby('TargetID')['TargetID', 'all VI issue', 'all VI comments']

In [13]:
truth_table=truth_table.merge(vi[['TargetID', 'all VI issue', 'all VI comments']], on='TargetID', how='left')

In [14]:
truth_table

,TargetID,Redrock z,Redrock spectype,best redshift,best spectype,vi_combined_flag,N_VI,DELTACHI2,FIBER,FLUX_G,...,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,merger comment,all VI issue,all VI comments
0,35191341453413620,0.3855,GALAXY,0.3855,GALAXY,4.0,3,651.904116,1252,2.533373,...,27.995237,1.009937,4.908875,11.160387,0.941849,0.960447,0.977679,none,--|--|--,--|--|--
1,35191341453413620,0.3855,GALAXY,0.3855,GALAXY,4.0,3,651.904116,1252,2.533373,...,27.995237,1.009937,4.908875,11.160387,0.941849,0.960447,0.977679,none,--|--|--,--|--|--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7743,35191366115918301,0.4056,GALAXY,0.4056,GALAXY,4.0,4,944.793370,2568,3.731254,...,47.370037,1.259283,6.676778,15.987193,0.956837,0.970716,0.983513,none,--|--|--|--,--|--|--| LRG
7744,35191366115918301,0.4056,GALAXY,0.4056,GALAXY,4.0,4,944.793370,2568,3.731254,...,47.370037,1.259283,6.676778,15.987193,0.956837,0.970716,0.983513,none,--|--|--|--,--|--|--| LRG


### Add more information from the FIBERMAP and ZBEST files

In [15]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

tspec_df = tspec['TARGETID','ZWARN', 'ZERR' ].to_pandas()
#tf_df = tf['TARGETID','MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z'].to_pandas()

#tf_df = tf_df.rename(columns={"TARGETID": "TargetID"})
tspec_df = tspec_df.rename(columns={"TARGETID": "TargetID"})

#truth_table = truth_table.merge(tf_df, how='left', on='TargetID')
truth_table = truth_table.merge(tspec_df, how='left', on='TargetID')

In [16]:
truth_table.keys()

Index(['TargetID', 'Redrock z', 'Redrock spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'N_VI', 'DELTACHI2', 'FIBER',
       'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R',
       'FIBERFLUX_Z', 'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R',
       'MW_TRANSMISSION_Z', 'merger comment', 'all VI issue',
       'all VI comments', 'ZWARN', 'ZERR'],
      dtype='object')

### Rename columns

* TargetID -> TARGETID
* best redshift -> best z
* vi_combined_flag -> best flag


In [17]:
truth_table = truth_table.rename(columns={"TargetID":"TARGETID", "best redshift":"best z", "vi_combined_flag":"best flag"})

In [18]:
truth_table.keys()

Index(['TARGETID', 'Redrock z', 'Redrock spectype', 'best z', 'best spectype',
       'best flag', 'N_VI', 'DELTACHI2', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z',
       'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'MW_TRANSMISSION_G',
       'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z', 'merger comment',
       'all VI issue', 'all VI comments', 'ZWARN', 'ZERR'],
      dtype='object')

### re-ordering columns

In [21]:
truth_table = truth_table[['TARGETID', 'Redrock z', 'Redrock spectype', 'best z', 'best spectype',
       'best flag', 'N_VI', 'all VI issue', 'all VI comments', 'merger comment', 'DELTACHI2', 'ZWARN', 'ZERR','FIBER', 
        'FLUX_G', 'FLUX_R', 'FLUX_Z',
       'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'MW_TRANSMISSION_G',
       'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z']]

In [22]:
truth_table.keys()

Index(['TARGETID', 'Redrock z', 'Redrock spectype', 'best z', 'best spectype',
       'best flag', 'N_VI', 'all VI issue', 'all VI comments',
       'merger comment', 'DELTACHI2', 'ZWARN', 'ZERR', 'FIBER', 'FLUX_G',
       'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z',
       'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z'],
      dtype='object')

In [23]:
print(VI_dir+save_filename)

/global/homes/r/rtojeiro/SV/VI_files/SV0/BGS/truth_table_BGS_v1.1.csv


In [24]:
truth_table.to_csv(VI_dir+save_filename)